<a href="https://colab.research.google.com/github/alyazone/Quranic-Theme-Extraction-Visualization/blob/main/ahmedali_refined_extractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('ahmedali-gpt-neo-2.7b-refined_themes.csv')

In [ ]:
verses_and_themes = [
    {"verse": row["Translation Verses"], "keywords": eval(row["Extracted Keywords"])}
    for _, row in df.iterrows()
]

In [ ]:
from transformers import pipeline

# initialize a pipeline for text generation
theme_extractor = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B", device=0)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
# Process in batches to avoid memory issues
batch_size = 1  # Adjust as necessary
all_refined_themes = []

for i in range(0, len(verses_and_themes), batch_size):
    batch = verses_and_themes[i:i+batch_size]
    print(f"Processing batch {i // batch_size + 1} out of {len(verses_and_themes) // batch_size + 1}")

    for entry in batch:

        try:
            input_text = f"Refine the themes for this verse: '{entry['verse']}' with keywords: {entry['keywords']}"
            with torch.no_grad():
                refined_themes = theme_extractor(input_text, max_new_tokens=10, num_return_sequences=1)
            all_refined_themes.append({
                "verse": entry["verse"],
                "original_keywords": entry["keywords"],
                "refined_themes": refined_themes[0]["generated_text"]
            })
        except Exception as e:
            print(f"Error processing entry: {entry}")
            print("Error message:", e)

# Convert refined themes to DataFrame for easier analysis and saving
refined_df = pd.DataFrame(all_refined_themes)
refined_df.to_csv("ahmedali-gpt-neo-1.3b-refined_themes.csv", index=False)

print("Refinement completed and saved to 'ahmedali-gpt-neo-1.3b-refined_themes.csv'")


Processing batch 1 out of 177
Error processing entry: {'verse': 'O MEN, FEAR your Lord who created you from a single cell, and from it created its mate, and from the two of them dispersed men and women (male and female) in multitudes. So fear God in whose name you ask of one another (the bond of) relationships. God surely keeps watch over you.', 'keywords': ['god surely keeps watch', 'fear god', 'whose name', 'single cell', 'one another', 'dispersed men', 'men', 'fear', 'women', 'two', 'relationships', 'multitudes', 'mate', 'male', 'lord', 'female', 'created', 'created', 'bond', 'ask']}
Error message: name 'torch' is not defined
Processing batch 2 out of 177
Error processing entry: {'verse': 'Give to the orphans their possessions, and do not replace things of your own which are bad with things which are good among theirs, and do not intermix their goods with your own and make use of them, for this is a grievous crime.', 'keywords': ['make use', 'grievous crime', 'good among', 'replace 